In [2]:
#pip install selenium

In [3]:
#pip install webdriver-manager

In [4]:
#pip install fake_useragent

In [32]:
# Import selenium packages

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from tqdm import tqdm

import numpy as np
import time

import pandas as pd

In [6]:
# Import webdriver

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

### Scrap Player's Ranks
Get the ranking of top 2k players

Page: https://www.atptour.com/en/rankings/singles?rankRange=1-5000&rankDate=2022-03-07

In [6]:
from fake_useragent import UserAgent

ua = UserAgent()
userAgent = ua.chrome

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument(f'user-agent={userAgent}')

browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/opt/anaconda3/lib/python3.9/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_mac64.zip
Driver has been saved in cache [/Users/mariahostiananapitupulu/.wdm/drivers/chromedriver/mac64/99.0.4844.51]
/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_4841/2052930819.py:13: DeprecationWarning: executable_path has been deprec

In [7]:
# Prepare file 

import csv

header = ['player', 'rank', 'age', 'player_point', 'tournament_played','next_best']

with open('tennis_ranking_data.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

In [8]:
# Open browser
browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
browser.get('https://www.atptour.com/en/rankings/singles?rankRange=1-5000&rankDate=2022-03-07')



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/mariahostiananapitupulu/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache
/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_4841/2602731997.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [9]:
# Run selenium to pull data

with open('tennis_ranking_data.csv', 'a', encoding='UTF8', newline='') as f:
    
    writer = csv.writer(f)
    
    for i in tqdm(range(1,2001)):

        player = browser.find_element(by=By.XPATH, value='//*[@id="player-rank-detail-ajax"]/tbody/tr['+str(i)+']/td[4]/span/a')
        rank = browser.find_element(by=By.XPATH, value='//*[@id="player-rank-detail-ajax"]/tbody/tr['+str(i)+']/td[1]')
        age = browser.find_element(by=By.XPATH, value='//*[@id="player-rank-detail-ajax"]/tbody/tr['+str(i)+']/td[5]')
        point = browser.find_element(by=By.XPATH, value='//*[@id="player-rank-detail-ajax"]/tbody/tr['+str(i)+']/td[6]/a')
        tourn_played = browser.find_element(by=By.XPATH, value='//*[@id="player-rank-detail-ajax"]/tbody/tr['+str(i)+']/td[7]/a')
        next_best = browser.find_element(by=By.XPATH, value='//*[@id="player-rank-detail-ajax"]/tbody/tr['+str(i)+']/td[9]')
        
        data=[player.text,rank.text,age.text,point.text,tourn_played.text,next_best.text]
        writer.writerow(data)

100%|███████████████████████████████████████| 2000/2000 [05:50<00:00,  5.70it/s]


In [10]:
# Store data into dataframe

import pandas as pd
df_ranking=pd.read_csv('tennis_ranking_data.csv')

In [11]:
# Assigning value for player's id
df_ranking["id_player"] = df_ranking.index + 1

In [12]:
df_ranking.head(10)

,player,rank,age,player_point,tournament_played,next_best,id_player
0,Daniil Medvedev,1,26.0,"8,615",23,0,1
1,Novak Djokovic,2,34.0,"8,465",13,0,2
2,Alexander Zverev,3,24.0,"7,515",24,10,3
3,Rafael Nadal,4,35.0,"6,515",11,0,4
4,Stefanos Tsitsipas,5,23.0,"6,325",27,45,5
5,Matteo Berrettini,6,25.0,"4,928",22,5,6
6,Andrey Rublev,7,24.0,"4,590",28,90,7
7,Casper Ruud,8,23.0,"3,915",31,90,8
8,Felix Auger-Aliassime,9,21.0,"3,883",25,45,9
9,Jannik Sinner,10,20.0,"3,495",38,44,10


In [13]:
# Drop row with empty value 
df_ranking = df_ranking.dropna(how = 'any', subset = ['age'])

In [14]:
# Remove decimal character
df_ranking['player_point'].replace(',','', regex=True, inplace=True)

In [15]:
# Creating temporary value for id_player 0 
temp = {'player': 'NaN','rank': 'NaN','age': 'NaN',
              'player_point': 'NaN','tournament_played': 'NaN','next_best': 'NaN',
              'id_player': '0',}

df_ranking = df_ranking.append(temp, ignore_index = True)

In [16]:
# Change datatype
df_ranking['id_player']= df_ranking['id_player'].astype(np.int64)

In [17]:
df_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   player             2000 non-null   object
 1   rank               2000 non-null   object
 2   age                2000 non-null   object
 3   player_point       2000 non-null   object
 4   tournament_played  2000 non-null   object
 5   next_best          2000 non-null   object
 6   id_player          2000 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 109.5+ KB


### Scrap Leaderboard

Page: https://www.atptour.com/en/stats/leaderboard?boardType=serve&timeFrame=Career&surface=all&versusRank=all&formerNo1=false

In [18]:
# Prepare file 

import csv

header = ['player', 'serve_rating', '1st_serve', '1st_serve_points_won', 
          '2nd_serve_points_won', 'service_games_won_percent','avg_aces_per_match','avg_double_faults_per_match']

with open('tennis_leaderboard_serve.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

In [19]:
from fake_useragent import UserAgent

ua = UserAgent()
userAgent = ua.chrome

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument(f'user-agent={userAgent}')

browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/mariahostiananapitupulu/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache
/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_4841/2052930819.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [20]:
# Open browser
browser.get('https://www.atptour.com/en/stats/leaderboard?boardType=serve&timeFrame=Career&surface=all&versusRank=all&formerNo1=false')

In [21]:
# Loop through the list 

with open('tennis_leaderboard_serve.csv', 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    for j in tqdm(range(1, 201)):
        player = browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(j)+']/td[2]/div/a')
        serve_rating = browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(j)+']/td[3]')
        serve_1st = browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(j)+']/td[4]')
        serve_won_1st = browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(j)+']/td[5]')
        serve_won_2nd = browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(j)+']/td[6]')
        service_games_won = browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(j)+']/td[7]')
        avg_aces_per_match = browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(j)+']/td[8]')
        avg_double_fault_per_match = browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(j)+']/td[9]')
         
        data=[player.text,serve_rating.text,serve_1st.text,serve_won_1st.text,serve_won_2nd.text,
              service_games_won.text,avg_aces_per_match.text,avg_double_fault_per_match.text]
        writer.writerow(data)

100%|█████████████████████████████████████████| 200/200 [00:40<00:00,  4.90it/s]


In [22]:
# Store data into dataframe

import pandas as pd
df_leaderboard_serve=pd.read_csv('tennis_leaderboard_serve.csv')

In [23]:
# Map id_player to leaderboard dataframe

df_subset=df_ranking[['id_player','player']]
df_leaderboard_mapped=pd.merge(df_leaderboard_serve,df_subset,on='player',how='left')
df_leaderboard_mapped

,player,serve_rating,1st_serve,1st_serve_points_won,2nd_serve_points_won,service_games_won_percent,avg_aces_per_match,avg_double_faults_per_match,id_player
0,John Isner,312.2,69.2%,78.7%,56.2%,91.9%,18.5,2.3,23.0
1,Ivo Karlovic,309.8,65.5%,82.7%,53.3%,92.0%,19.8,3.5,336.0
2,Reilly Opelka,304.9,64.1%,79.4%,55.1%,90.5%,18.7,2.9,17.0
3,Milos Raonic,302.8,62.9%,81.4%,55.1%,91.2%,15.5,3.3,156.0
4,Andy Roddick,299.9,64.8%,79.3%,56.0%,90.1%,11.7,2.0,NaN
...,...,...,...,...,...,...,...,...,...
195,Robert Kendrick,263.4,58.4%,74.0%,47.4%,78.3%,9.6,4.3,NaN
196,Michael Berrer,263.3,64.1%,70.9%,48.4%,77.6%,5.7,3.4,NaN
197,Marcos Baghdatis,263.3,53.1%,74.8%,51.5%,79.9%,7.3,3.3,NaN
198,Brian Dabul,263.3,68.0%,67.8%,50.3%,76.6%,2.3,1.7,NaN


In [24]:
# Fill 0 to unmatch values
df_leaderboard_mapped['id_player']= df_leaderboard_mapped['id_player'].fillna(0)
df_leaderboard_mapped['id_player']= df_leaderboard_mapped['id_player'].astype(np.int64)

In [25]:
# Assigning value for player's id
df_leaderboard_mapped["id_leaderboard"] = df_leaderboard_mapped.index + 1

In [26]:
# Removing unwanted character
df_leaderboard_mapped['1st_serve'] = df_leaderboard_mapped['1st_serve'].str.replace('%', '')
df_leaderboard_mapped['1st_serve_points_won'] = df_leaderboard_mapped['1st_serve_points_won'].str.replace('%', '')
df_leaderboard_mapped['2nd_serve_points_won'] = df_leaderboard_mapped['2nd_serve_points_won'].str.replace('%', '')
df_leaderboard_mapped['service_games_won_percent'] = df_leaderboard_mapped['service_games_won_percent'].str.replace('%', '')

In [27]:
df_leaderboard_mapped['1st_serve']= df_leaderboard_mapped['1st_serve'].astype(np.float64)
df_leaderboard_mapped['1st_serve_points_won']= df_leaderboard_mapped['1st_serve_points_won'].astype(np.float64)
df_leaderboard_mapped['2nd_serve_points_won']= df_leaderboard_mapped['2nd_serve_points_won'].astype(np.float64)
df_leaderboard_mapped['service_games_won_percent']= df_leaderboard_mapped['service_games_won_percent'].astype(np.float64)

In [28]:
df_leaderboard_mapped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player                       200 non-null    object 
 1   serve_rating                 200 non-null    float64
 2   1st_serve                    200 non-null    float64
 3   1st_serve_points_won         200 non-null    float64
 4   2nd_serve_points_won         200 non-null    float64
 5   service_games_won_percent    200 non-null    float64
 6   avg_aces_per_match           200 non-null    float64
 7   avg_double_faults_per_match  200 non-null    float64
 8   id_player                    200 non-null    int64  
 9   id_leaderboard               200 non-null    int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 17.2+ KB


### Scrap Tournament Page

Page: https://www.atptour.com/en/scores/results-archive?year=2022'

In [29]:
from fake_useragent import UserAgent

ua = UserAgent()
userAgent = ua.chrome

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument(f'user-agent={userAgent}')

browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/mariahostiananapitupulu/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache
/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_4841/2052930819.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [30]:
# Open browser
browser.get('https://www.atptour.com/en/scores/results-archive?year=2022')

In [31]:
import csv

header = ['tournament_name', 'tournament_location','tournament_date',
          'tournament_draw', 'tournament_surface', 'tournament_financial_commitment']

with open('tournament_details_scrape.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

In [32]:
all_offers = browser.find_elements_by_class_name("tourney-result")

data = []

for offer in all_offers:
    # uses `offer` instead `driver` to search only in this one offer
    # uses `element` instead of `elements` to search only one value

    try:
        title_content = offer.find_element_by_class_name("title-content").text.strip()
    except Exception as ex:
        #print('[Exception] company:', ex)
        title_content = 'NAN'

    tournament_name = offer.find_element_by_class_name("tourney-title").text.strip()
    tournament_location = offer.find_element_by_class_name("tourney-location").text.strip()
    tournament_date = offer.find_element_by_class_name("tourney-dates").text.strip()
    #tournament_draw = offer.find_element_by_class_name("item-details").text.strip()
    #tournament_surface = offer.find_element_by_class_name("tourney-details").text.strip()
    tournament_financial_commitment = offer.find_element_by_class_name("tourney-details.fin-commit").text.strip()
    
    #looping through the tourney_detail class to get each of tournament draw and surface
    #as these have same class names and we were unable to get values by using class names
    #as done above for other variables.
    all_offers1 = browser.find_elements_by_class_name("tourney-details")
    count=0
    for offer1 in all_offers1:
        count=count+1
        if count==1:
            tournament_draw = offer1.text
        elif count ==2:
            tournament_surface = offer1.text
        else:
            break
            
            
    data.append([tournament_name,tournament_location, tournament_date, tournament_draw, tournament_surface,tournament_financial_commitment])
    
with open('tournament_details_scrape.csv', 'a', encoding='UTF8', newline='') as f:
    #for row in data:
    write = csv.writer(f)
    write.writerows(data)    

/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_4841/3563184211.py:1: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  all_offers = browser.find_elements_by_class_name("tourney-result")
/opt/anaconda3/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:446: UserWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  warnings.warn("find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead")
/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_4841/3563184211.py:25: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  all_offers1 = browser.find_elements_by_class_name("tourney-details")


In [33]:
df_tournament = pd.read_csv('tournament_details_scrape.csv')

In [34]:
# Assigning value for id_tournament
df_tournament["id_tournament"] = df_tournament.index + 1

In [35]:
df_tournament.head(10)

,tournament_name,tournament_location,tournament_date,tournament_draw,tournament_surface,tournament_financial_commitment,id_tournament
0,ATP Cup,"Sydney, Australia",2022.01.01,SGL 16 DBL 16,Outdoor Hard,"$10,000,000",1
1,Adelaide International 1,"Adelaide, Australia",2022.01.03,SGL 16 DBL 16,Outdoor Hard,"$416,800",2
2,Melbourne Summer Set,"Melbourne, Australia",2022.01.04,SGL 16 DBL 16,Outdoor Hard,"$521,000",3
3,Sydney Tennis Classic,"Sydney, Australia",2022.01.10,SGL 16 DBL 16,Outdoor Hard,"$521,000",4
4,Adelaide International 2,"Adelaide, Australia",2022.01.10,SGL 16 DBL 16,Outdoor Hard,"$493,875",5
5,Australian Open,"Melbourne, Australia",2022.01.17,SGL 16 DBL 16,Outdoor Hard,"A$33,784,200",6
6,Cordoba Open,"Cordoba, Argentina",2022.01.31,SGL 16 DBL 16,Outdoor Hard,"$493,875",7
7,Tata Open Maharashtra,"Pune, India",2022.01.31,SGL 16 DBL 16,Outdoor Hard,"$493,875",8
8,Open Sud de France – Montpellier,"Montpellier, France",2022.01.31,SGL 16 DBL 16,Outdoor Hard,"€490,990",9
9,ABN Amro World Tennis Tournament,"Rotterdam, Netherlands",2022.02.07,SGL 16 DBL 16,Outdoor Hard,"€1,349,070",10


In [36]:
df_tournament.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   tournament_name                  70 non-null     object
 1   tournament_location              70 non-null     object
 2   tournament_date                  70 non-null     object
 3   tournament_draw                  70 non-null     object
 4   tournament_surface               70 non-null     object
 5   tournament_financial_commitment  38 non-null     object
 6   id_tournament                    70 non-null     int64 
dtypes: int64(1), object(6)
memory usage: 4.0+ KB


### Scrap Match Single

Page: https://www.atptour.com/en/scores/current/indian-wells/404/results?matchType=singles

In [37]:
from fake_useragent import UserAgent

ua = UserAgent()
userAgent = ua.chrome

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument(f'user-agent={userAgent}')

browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/mariahostiananapitupulu/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache
/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_4841/2052930819.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [38]:
# Open web page
web_single = 'https://www.atptour.com/en/scores/current/indian-wells/404/results?matchType=singles'

In [39]:
# Open browser single
browser.get(web_single)

In [40]:
# Define class for match score
all_data = browser.find_element(by=By.XPATH, value='/html/body/div[3]/div[2]/div[1]/div/div[5]/div/table')
players = all_data.find_elements(by=By.CLASS_NAME, value='day-table-name')
scores = all_data.find_elements(by=By.CLASS_NAME, value='day-table-score')

In [41]:
# Create dataframe for player's details
match_player = dict()
match_score = dict()

match_player_single = pd.DataFrame()
match_score_single = pd.DataFrame()

In [42]:
# Fetch players & scores data for single match
    
for player in players:
    match_player['player']=player.text
    match_player_single = match_player_single.append(match_player, ignore_index=True)
            
for score in scores: 
    match_score['score']=score.text
    match_score_single = match_score_single.append(match_score, ignore_index=True)

In [43]:
# Player 1 and Player 2 data has similar class. Thus, we need to split even & odd row into two different column.
match_player_single

,player
0,Taylor Fritz
1,Rafael Nadal
2,Rafael Nadal
3,Carlos Alcaraz
4,Taylor Fritz
...,...
257,Alex Bolt
258,J.J. Wolf
259,Emilio Gomez
260,Tennys Sandgren


In [44]:
# Split players data into player 1 & player 2  
player_1=match_player_single.iloc[::2]
player_2=match_player_single.iloc[1::2] 
player_1=player_1.reset_index(drop=True)
player_2=player_2.reset_index(drop=True)

# Rename column
player_1.rename(columns = {'player':'player_1'}, inplace = True)
player_2.rename(columns = {'player':'player_2'}, inplace = True)

In [45]:
# Concat all columns
merge_df=pd.concat([player_1, player_2,match_score_single], axis=1)

# Rename column
merge_df.rename(columns = {'player':'player_1'}, inplace = True)

In [46]:
# Match data overview
merge_df.head(10)

,player_1,player_2,score
0,Taylor Fritz,Rafael Nadal,63 765
1,Rafael Nadal,Carlos Alcaraz,64 46 63
2,Taylor Fritz,Andrey Rublev,75 64
3,Rafael Nadal,Nick Kyrgios,760 57 64
4,Andrey Rublev,Grigor Dimitrov,75 62
5,Carlos Alcaraz,Cameron Norrie,64 63
6,Taylor Fritz,Miomir Kecmanovic,765 36 61
7,Rafael Nadal,Reilly Opelka,763 765
8,Miomir Kecmanovic,Matteo Berrettini,63 675 64
9,Andrey Rublev,Hubert Hurkacz,765 64


The score value is still in one line for all sets. Thus, we will separate them into several columns.

In [47]:
# Split score into columns 
score_split=merge_df['score'].str.split(' ', expand=True)

In [48]:
# Merge back score values
match_single_df=pd.concat([merge_df,score_split], axis=1)
match_single_df.rename(columns = {0:'set_1',1:'set_2',2:'set_3'}, inplace = True)

In [49]:
match_single_df

,player_1,player_2,score,set_1,set_2,set_3
0,Taylor Fritz,Rafael Nadal,63 765,63,765,None
1,Rafael Nadal,Carlos Alcaraz,64 46 63,64,46,63
2,Taylor Fritz,Andrey Rublev,75 64,75,64,None
3,Rafael Nadal,Nick Kyrgios,760 57 64,760,57,64
4,Andrey Rublev,Grigor Dimitrov,75 62,75,62,None
...,...,...,...,...,...,...
126,Vasek Pospisil,Egor Gerasimov,46 64 64,46,64,64
127,Daniel Masur,Elias Ymer,64 75,64,75,None
128,Philipp Kohlschreiber,Alex Bolt,26 64 64,26,64,64
129,J.J. Wolf,Emilio Gomez,61 63,61,63,None


In [50]:
match_single_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   player_1  131 non-null    object
 1   player_2  131 non-null    object
 2   score     131 non-null    object
 3   set_1     131 non-null    object
 4   set_2     129 non-null    object
 5   set_3     47 non-null     object
dtypes: object(6)
memory usage: 6.3+ KB


Next, each set's score will be calculated to define which player wins the match. 

The score value is still in one line for all sets. Thus, we will separate them into several columns.

In [51]:
# Split score into columns 
score_split=merge_df['score'].str.split(' ', expand=True)

# Merge back score values
match_single_df=pd.concat([merge_df,score_split], axis=1)
match_single_df.rename(columns = {0:'set_1',1:'set_2',2:'set_3'}, inplace = True)

In [52]:
match_single_df

,player_1,player_2,score,set_1,set_2,set_3
0,Taylor Fritz,Rafael Nadal,63 765,63,765,None
1,Rafael Nadal,Carlos Alcaraz,64 46 63,64,46,63
2,Taylor Fritz,Andrey Rublev,75 64,75,64,None
3,Rafael Nadal,Nick Kyrgios,760 57 64,760,57,64
4,Andrey Rublev,Grigor Dimitrov,75 62,75,62,None
...,...,...,...,...,...,...
126,Vasek Pospisil,Egor Gerasimov,46 64 64,46,64,64
127,Daniel Masur,Elias Ymer,64 75,64,75,None
128,Philipp Kohlschreiber,Alex Bolt,26 64 64,26,64,64
129,J.J. Wolf,Emilio Gomez,61 63,61,63,None


In [53]:
# Map id_player to match dataframe
df_match_single_mapped=pd.merge(match_single_df,df_subset,left_on='player_1',right_on='player',how='left')
df_match_single_mapped.rename(columns = {'id_player':'id_player_1'}, inplace = True)

df_match_single_mapped=pd.merge(df_match_single_mapped,df_subset,left_on='player_2',right_on='player',how='left')
df_match_single_mapped.rename(columns = {'id_player':'id_player_2'}, inplace = True)

df_match_single_mapped.drop(columns=['player_x', 'player_y'],inplace=True)

In [54]:
# Change datatype
df_match_single_mapped['id_player_1']= df_match_single_mapped['id_player_1'].fillna(0)
df_match_single_mapped['id_player_2']= df_match_single_mapped['id_player_2'].fillna(0)

df_match_single_mapped['id_player_1']= df_match_single_mapped['id_player_1'].astype(np.int64)
df_match_single_mapped['id_player_2']= df_match_single_mapped['id_player_2'].astype(np.int64)

In [55]:
df_match_single_mapped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131 entries, 0 to 130
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   player_1     131 non-null    object
 1   player_2     131 non-null    object
 2   score        131 non-null    object
 3   set_1        131 non-null    object
 4   set_2        129 non-null    object
 5   set_3        47 non-null     object
 6   id_player_1  131 non-null    int64 
 7   id_player_2  131 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 9.2+ KB


Next, each set's score will be calculated to define which player wins the match. 

### Scrap Match Double
The scores scrapped until 18 March 2022

Page: https://www.atptour.com/en/scores/current/indian-wells/404/results?matchType=doubles

In [56]:
from fake_useragent import UserAgent

ua = UserAgent()
userAgent = ua.chrome

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument(f'user-agent={userAgent}')

browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/mariahostiananapitupulu/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache
/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_4841/2052930819.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [57]:
# Open web page
web_double = 'https://www.atptour.com/en/scores/current/indian-wells/404/results?matchType=doubles'

In [58]:
# Open browser double
browser.get(web_double)

In [59]:
# Define class for match
all_data = browser.find_element(by=By.XPATH, value='/html/body/div[3]/div[2]/div[1]/div/div[5]/div/table')
players = all_data.find_elements(by=By.CLASS_NAME, value='day-table-name')
scores = all_data.find_elements(by=By.CLASS_NAME, value='day-table-score')

In [60]:
# Create dataframe
match_player=dict()
match_score=dict()
match_player_double = pd.DataFrame()
match_score_double = pd.DataFrame()

In [61]:
# Fetch players & scores data for single match

for player in players:
    match_player['player']=player.text
    match_player_double = match_player_double.append(match_player, ignore_index=True)
    
for score in scores: 
    match_score['score']=score.text
    match_score_double = match_score_double.append(match_score, ignore_index=True)

In [62]:
# The 2 players played appear in the same cell. Thus, we need to split them into 2 columns first, then split the even & odd row into two different column.
match_player_double.head(10)

,player
0,John Isner\nJack Sock
1,Santiago Gonzalez\nEdouard Roger-Vasselin
2,Santiago Gonzalez\nEdouard Roger-Vasselin
3,Rajeev Ram\nJoe Salisbury
4,John Isner\nJack Sock
5,Andrey Golubev\nAlexander Zverev
6,Rajeev Ram\nJoe Salisbury
7,Wesley Koolhof\nNeal Skupski
8,Andrey Golubev\nAlexander Zverev
9,Taylor Fritz\nTommy Paul


In [63]:
# Split player into two columns 
match_player_double=match_player_double['player'].str.split('\n', expand=True)

In [64]:
# Rename column
match_player_double.rename(columns = {0:'t1_player_1',1:'t1_player_2'}, inplace = True)
match_player_double.head(10)

,t1_player_1,t1_player_2
0,John Isner,Jack Sock
1,Santiago Gonzalez,Edouard Roger-Vasselin
2,Santiago Gonzalez,Edouard Roger-Vasselin
3,Rajeev Ram,Joe Salisbury
4,John Isner,Jack Sock
5,Andrey Golubev,Alexander Zverev
6,Rajeev Ram,Joe Salisbury
7,Wesley Koolhof,Neal Skupski
8,Andrey Golubev,Alexander Zverev
9,Taylor Fritz,Tommy Paul


In [65]:
# Split players data into player 1 & player 2  
t1=match_player_double.iloc[::2]
t2=match_player_double.iloc[1::2] 

t1=t1.reset_index(drop=True)
t2=t2.reset_index(drop=True)

# Rename column for team 2
t2.rename(columns = {'t1_player_1':'t2_player_1','t1_player_2':'t2_player_2'}, inplace = True)

In [66]:
# Concat all columns
merge_double_df=pd.concat([t1, t2,match_score_double], axis=1)

# Rename column
#merge_df.rename(columns = {'player':'player_1'}, inplace = True)

In [67]:
# Match data overview
merge_double_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   t1_player_1  31 non-null     object
 1   t1_player_2  31 non-null     object
 2   t2_player_1  31 non-null     object
 3   t2_player_2  31 non-null     object
 4   score        31 non-null     object
dtypes: object(5)
memory usage: 1.3+ KB


The score value is still in one line for all sets. Thus, we will separate them into several columns.

In [68]:
# Split score into columns 
score_split=merge_double_df['score'].str.split(' ', expand=True)

In [69]:
# Merge back score values
match_double_df=pd.concat([merge_double_df,score_split], axis=1)
match_double_df.rename(columns = {0:'set_1',1:'set_2',2:'set_3'}, inplace = True)

In [70]:
# Map id_player to match dataframe
df_match_double_mapped=pd.merge(match_double_df,df_subset,left_on='t1_player_1',right_on='player',how='left')
df_match_double_mapped.rename(columns = {'id_player':'id_player_t1_1'}, inplace = True)

df_match_double_mapped=pd.merge(df_match_double_mapped,df_subset,left_on='t1_player_2',right_on='player',how='left')
df_match_double_mapped.rename(columns = {'id_player':'id_player_t1_2'}, inplace = True)

df_match_double_mapped=pd.merge(df_match_double_mapped,df_subset,left_on='t2_player_1',right_on='player',how='left')
df_match_double_mapped.rename(columns = {'id_player':'id_player_t2_1'}, inplace = True)

df_match_double_mapped=pd.merge(df_match_double_mapped,df_subset,left_on='t2_player_2',right_on='player',how='left')
df_match_double_mapped.rename(columns = {'id_player':'id_player_t2_2'}, inplace = True)

df_match_double_mapped.drop(columns=['player_x', 'player_y'],inplace=True)

/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_4841/1316321303.py:11: FutureWarning: Passing 'suffixes' which cause duplicate columns {'player_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_match_double_mapped=pd.merge(df_match_double_mapped,df_subset,left_on='t2_player_2',right_on='player',how='left')


In [71]:
df_match_double_mapped.head(10)

,t1_player_1,t1_player_2,t2_player_1,t2_player_2,score,set_1,set_2,set_3,id_player_t1_1,id_player_t1_2,id_player_t2_1,id_player_t2_2
0,John Isner,Jack Sock,Santiago Gonzalez,Edouard Roger-Vasselin,764 63,764,63,None,23.0,147.0,NaN,NaN
1,Santiago Gonzalez,Edouard Roger-Vasselin,Rajeev Ram,Joe Salisbury,36 64 10-7,36,64,10-7,NaN,NaN,NaN,NaN
2,John Isner,Jack Sock,Andrey Golubev,Alexander Zverev,63 62,63,62,None,23.0,147.0,947.0,3.0
3,Rajeev Ram,Joe Salisbury,Wesley Koolhof,Neal Skupski,764 75,764,75,None,NaN,NaN,NaN,NaN
4,Andrey Golubev,Alexander Zverev,Taylor Fritz,Tommy Paul,64 763,64,763,None,947.0,3.0,20.0,39.0
5,Santiago Gonzalez,Edouard Roger-Vasselin,Aslan Karatsev,Andrey Rublev,62 64,62,64,None,NaN,NaN,22.0,7.0
6,John Isner,Jack Sock,Feliciano Lopez,Stefanos Tsitsipas,75 64,75,64,None,23.0,147.0,112.0,5.0
7,Rajeev Ram,Joe Salisbury,Andres Molteni,Diego Schwartzman,63 762,63,762,None,NaN,NaN,NaN,14.0
8,Aslan Karatsev,Andrey Rublev,Marcel Granollers,Horacio Zeballos,762 46 10-4,762,46,10-4,22.0,7.0,301.0,NaN
9,Taylor Fritz,Tommy Paul,Juan Sebastian Cabal,Robert Farah,61 64,61,64,None,20.0,39.0,NaN,NaN


In [72]:
df_match_double_mapped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 30
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   t1_player_1     31 non-null     object 
 1   t1_player_2     31 non-null     object 
 2   t2_player_1     31 non-null     object 
 3   t2_player_2     31 non-null     object 
 4   score           31 non-null     object 
 5   set_1           31 non-null     object 
 6   set_2           31 non-null     object 
 7   set_3           8 non-null      object 
 8   id_player_t1_1  19 non-null     float64
 9   id_player_t1_2  18 non-null     float64
 10  id_player_t2_1  18 non-null     float64
 11  id_player_t2_2  15 non-null     float64
dtypes: float64(4), object(8)
memory usage: 3.1+ KB


In [73]:
# Fill NaN data and change datatype
df_match_double_mapped['id_player_t1_1']= df_match_double_mapped['id_player_t1_1'].fillna(0)
df_match_double_mapped['id_player_t1_2']= df_match_double_mapped['id_player_t1_2'].fillna(0)
df_match_double_mapped['id_player_t2_1']= df_match_double_mapped['id_player_t2_1'].fillna(0)
df_match_double_mapped['id_player_t2_2']= df_match_double_mapped['id_player_t2_2'].fillna(0)

df_match_double_mapped['id_player_t1_1']= df_match_double_mapped['id_player_t1_1'].astype(np.int64)
df_match_double_mapped['id_player_t1_1']= df_match_double_mapped['id_player_t1_2'].astype(np.int64)
df_match_double_mapped['id_player_t2_1']= df_match_double_mapped['id_player_t2_1'].astype(np.int64)
df_match_double_mapped['id_player_t2_2']= df_match_double_mapped['id_player_t2_2'].astype(np.int64)

In [74]:
df_match_double_mapped.head(10)

,t1_player_1,t1_player_2,t2_player_1,t2_player_2,score,set_1,set_2,set_3,id_player_t1_1,id_player_t1_2,id_player_t2_1,id_player_t2_2
0,John Isner,Jack Sock,Santiago Gonzalez,Edouard Roger-Vasselin,764 63,764,63,None,147,147.0,0,0
1,Santiago Gonzalez,Edouard Roger-Vasselin,Rajeev Ram,Joe Salisbury,36 64 10-7,36,64,10-7,0,0.0,0,0
2,John Isner,Jack Sock,Andrey Golubev,Alexander Zverev,63 62,63,62,None,147,147.0,947,3
3,Rajeev Ram,Joe Salisbury,Wesley Koolhof,Neal Skupski,764 75,764,75,None,0,0.0,0,0
4,Andrey Golubev,Alexander Zverev,Taylor Fritz,Tommy Paul,64 763,64,763,None,3,3.0,20,39
5,Santiago Gonzalez,Edouard Roger-Vasselin,Aslan Karatsev,Andrey Rublev,62 64,62,64,None,0,0.0,22,7
6,John Isner,Jack Sock,Feliciano Lopez,Stefanos Tsitsipas,75 64,75,64,None,147,147.0,112,5
7,Rajeev Ram,Joe Salisbury,Andres Molteni,Diego Schwartzman,63 762,63,762,None,0,0.0,0,14
8,Aslan Karatsev,Andrey Rublev,Marcel Granollers,Horacio Zeballos,762 46 10-4,762,46,10-4,7,7.0,301,0
9,Taylor Fritz,Tommy Paul,Juan Sebastian Cabal,Robert Farah,61 64,61,64,None,39,39.0,0,0


In [75]:
# Assign match title & price to main dataset
df_match_single_mapped = df_match_single_mapped.assign(id_tournament='19')
df_match_double_mapped = df_match_double_mapped.assign(id_tournament='19')

In [76]:
df_match_single_mapped

,player_1,player_2,score,set_1,set_2,set_3,id_player_1,id_player_2,id_tournament
0,Taylor Fritz,Rafael Nadal,63 765,63,765,None,20,4,19
1,Rafael Nadal,Carlos Alcaraz,64 46 63,64,46,63,4,19,19
2,Taylor Fritz,Andrey Rublev,75 64,75,64,None,20,7,19
3,Rafael Nadal,Nick Kyrgios,760 57 64,760,57,64,4,132,19
4,Andrey Rublev,Grigor Dimitrov,75 62,75,62,None,7,35,19
...,...,...,...,...,...,...,...,...,...
126,Vasek Pospisil,Egor Gerasimov,46 64 64,46,64,64,129,148,19
127,Daniel Masur,Elias Ymer,64 75,64,75,None,176,131,19
128,Philipp Kohlschreiber,Alex Bolt,26 64 64,26,64,64,136,188,19
129,J.J. Wolf,Emilio Gomez,61 63,61,63,None,209,141,19


In [77]:
# Change datatype
df_match_single_mapped['id_player_1']= df_match_single_mapped['id_player_1'].astype(np.int64)
df_match_single_mapped['id_tournament']= df_match_single_mapped['id_tournament'].astype(np.int64)
df_match_double_mapped['id_tournament']= df_match_double_mapped['id_tournament'].astype(np.int64)
df_match_double_mapped['id_player_t1_2']= df_match_double_mapped['id_player_t1_2'].astype(np.int64)

In [78]:
# Assigning value for id_match
df_match_single_mapped["id_match"] = df_match_single_mapped.index + 1
df_match_double_mapped["id_match"] = df_match_double_mapped.index + 1

In [79]:
df_match_single_mapped

,player_1,player_2,score,set_1,set_2,set_3,id_player_1,id_player_2,id_tournament,id_match
0,Taylor Fritz,Rafael Nadal,63 765,63,765,None,20,4,19,1
1,Rafael Nadal,Carlos Alcaraz,64 46 63,64,46,63,4,19,19,2
2,Taylor Fritz,Andrey Rublev,75 64,75,64,None,20,7,19,3
3,Rafael Nadal,Nick Kyrgios,760 57 64,760,57,64,4,132,19,4
4,Andrey Rublev,Grigor Dimitrov,75 62,75,62,None,7,35,19,5
...,...,...,...,...,...,...,...,...,...,...
126,Vasek Pospisil,Egor Gerasimov,46 64 64,46,64,64,129,148,19,127
127,Daniel Masur,Elias Ymer,64 75,64,75,None,176,131,19,128
128,Philipp Kohlschreiber,Alex Bolt,26 64 64,26,64,64,136,188,19,129
129,J.J. Wolf,Emilio Gomez,61 63,61,63,None,209,141,19,130


In [80]:
df_match_single_mapped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131 entries, 0 to 130
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   player_1       131 non-null    object
 1   player_2       131 non-null    object
 2   score          131 non-null    object
 3   set_1          131 non-null    object
 4   set_2          129 non-null    object
 5   set_3          47 non-null     object
 6   id_player_1    131 non-null    int64 
 7   id_player_2    131 non-null    int64 
 8   id_tournament  131 non-null    int64 
 9   id_match       131 non-null    int64 
dtypes: int64(4), object(6)
memory usage: 11.3+ KB


In [81]:
df_match_double_mapped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 30
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   t1_player_1     31 non-null     object
 1   t1_player_2     31 non-null     object
 2   t2_player_1     31 non-null     object
 3   t2_player_2     31 non-null     object
 4   score           31 non-null     object
 5   set_1           31 non-null     object
 6   set_2           31 non-null     object
 7   set_3           8 non-null      object
 8   id_player_t1_1  31 non-null     int64 
 9   id_player_t1_2  31 non-null     int64 
 10  id_player_t2_1  31 non-null     int64 
 11  id_player_t2_2  31 non-null     int64 
 12  id_tournament   31 non-null     int64 
 13  id_match        31 non-null     int64 
dtypes: int64(6), object(8)
memory usage: 3.6+ KB


Saving all scrapped file into csv

In [82]:
# Write into csv file 
df_ranking.to_csv(r'/Users/mariahostiananapitupulu/Documents/Term 2/Data Engineering/Group Coursework/DE_files_new/atp_rank_scrapped.csv',
                  index=False, decimal=",")

df_leaderboard_mapped.to_csv(r'/Users/mariahostiananapitupulu/Documents/Term 2/Data Engineering/Group Coursework/DE_files_new/atp_leaderboard_scrapped.csv',
                  index=False, decimal=".")

df_tournament.to_csv(r'/Users/mariahostiananapitupulu/Documents/Term 2/Data Engineering/Group Coursework/DE_files_new/atp_tournament_scrapped.csv',
                  index=False, decimal=",")

df_match_single_mapped.to_csv(r'/Users/mariahostiananapitupulu/Documents/Term 2/Data Engineering/Group Coursework/DE_files_new/match_single_scrapped.csv', 
                       sep = ';', index=False, decimal=",")

df_match_double_mapped.to_csv(r'/Users/mariahostiananapitupulu/Documents/Term 2/Data Engineering/Group Coursework/DE_files_new/match_double_scrapped.csv', 
                       sep = ';', index=False, decimal=",")

## TRIAL

Players
- get more features
- add WTA (?)

Betting
- direct source the data from http://tennis-data.co.uk
    
Match 
- get from betting 

Twitter
- only if someone wants to do sentiment analysis

#### Scrape win/loss stat of the player's performance career overall from all countries

In [140]:
from fake_useragent import UserAgent

ua = UserAgent()
userAgent = ua.chrome

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--no-sandbox")
#chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument(f'user-agent={userAgent}')

browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/mariahostiananapitupulu/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_1423/2020879156.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [141]:
# Open web page
web_win_loss = 'https://www.atptour.com/en/stats/win-loss-index/career/all/all/'

In [142]:
# Open browser double
browser.get(web_win_loss)

In [143]:
# Define class for match

all_data=browser.find_element(by=By.XPATH, value='/html/body/div[3]/div[2]/div[1]/div/div[4]/div/div/div/div')
players=all_data.find_elements(by=By.CLASS_NAME, value='player-cell')
career_idxs=all_data.find_elements(by=By.CLASS_NAME, value='fifty-two-week-index-cell')
career_titles=all_data.find_elements(by=By.CLASS_NAME, value='fifty-two-week-titles-cell')
career_win_losses=all_data.find_elements(by=By.CLASS_NAME, value='fifty-two-week-win-loss-cell')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[2]/div[1]/div/div[4]/div/div/div/div"}
  (Session info: chrome=100.0.4896.75)
Stacktrace:
0   chromedriver                        0x0000000100cb33d9 chromedriver + 5104601
1   chromedriver                        0x0000000100c43bf3 chromedriver + 4647923
2   chromedriver                        0x0000000100834b48 chromedriver + 392008
3   chromedriver                        0x0000000100869e26 chromedriver + 609830
4   chromedriver                        0x0000000100869fe1 chromedriver + 610273
5   chromedriver                        0x000000010089c374 chromedriver + 815988
6   chromedriver                        0x00000001008876ed chromedriver + 730861
7   chromedriver                        0x000000010089a091 chromedriver + 807057
8   chromedriver                        0x00000001008875b3 chromedriver + 730547
9   chromedriver                        0x000000010

In [ ]:
# Create dataframe
player_dict=dict()
career_idx_dict=dict()
career_title_dict=dict()
career_win_loss_dict=dict()

player_df = pd.DataFrame()
career_idx_df = pd.DataFrame()
career_title_df = pd.DataFrame()
career_win_loss_df = pd.DataFrame()

In [ ]:
# Fetch all data

for player in players:
    player_dict['player']=player.text
    player_df = player_df.append(player_dict, ignore_index=True)

for career_idx in career_idxs:
    career_idx_dict['career_idx']=career_idx.text
    career_idx_df = career_idx_df.append(career_idx_dict, ignore_index=True)
    
for career_title in career_titles:
    career_title_dict['career_title']=career_title.text
    career_title_df = career_title_df.append(career_title_dict, ignore_index=True)
    
for career_win_loss in career_win_losses:
    career_win_loss_dict['career_win_loss']=career_win_loss.text
    career_win_loss_df = career_win_loss_df.append(career_win_loss_dict, ignore_index=True)

In [ ]:
# Concat all columns
merge_df=pd.concat([player_df,career_idx_df,career_title_df,career_win_loss_df], axis=1)

In [ ]:
merge_df

#### Scrape leaderboard stats (serve leader, return leader, and under pressure leader) 

Serve Leaders Versus All Players On All Surfaces For Career

In [152]:
from fake_useragent import UserAgent

ua = UserAgent()
userAgent = ua.chrome

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument(f'user-agent={userAgent}')

browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/mariahostiananapitupulu/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_1423/2052930819.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [153]:
# Define web pages
leaderboard_serve_leader = 'https://www.atptour.com/en/stats/leaderboard?boardType=serve&timeFrame=Career&surface=all&versusRank=all&formerNo1=false'
leaderboard_pressure_leader = 'https://www.atptour.com/en/stats/leaderboard?boardType=pressure&timeFrame=Career&surface=all&versusRank=all&formerNo1=false'

In [154]:
# Open browser serve leaderboard
browser.get(leaderboard_serve_leader)

In [155]:
# Define class for serve leaderboard
all_data=browser.find_element(by=By.XPATH, value='/html/body/div[3]/div[2]/div[1]/div/div[1]/div[2]/div[3]/div/div[1]/div/table')
players=all_data.find_elements(by=By.CLASS_NAME, value='stats-player-name')

In [156]:
# Create dataframe
player_dict=dict()
serve_rating_idx_dict=dict()
rate_1st_serve_dict=dict()
rate_1st_serve_points_dict=dict()
rate_2nd_serve_points_dict=dict()
rate_service_game_won_dict=dict()
avg_aces_per_match_dict=dict()
avg_double_faults_per_match_dict=dict()

player_df = pd.DataFrame()
serve_rating_df = pd.DataFrame()
rate_1st_serve_df=pd.DataFrame()
rate_1st_serve_points_df=pd.DataFrame()
rate_2nd_serve_points_df=pd.DataFrame()
rate_service_game_won_df=pd.DataFrame()
avg_aces_per_match_df=pd.DataFrame()
avg_double_faults_per_match_df=pd.DataFrame()

In [157]:
# Fetch all data

i=1

for player in players:
    player_dict['player']=player.text
    player_df = player_df.append(player_dict, ignore_index=True)
    
    serve_rating=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[3]')
    serve_rating_idx_dict['serve_rating']=serve_rating.text
    serve_rating_df = serve_rating_df.append(serve_rating_idx_dict, ignore_index=True)

    rate_1st_serve=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[4]')
    rate_1st_serve_dict['rate_1st_serve']=rate_1st_serve.text
    rate_1st_serve_df = rate_1st_serve_df.append(rate_1st_serve_dict, ignore_index=True)

    rate_1st_serve_points=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[5]')
    rate_1st_serve_points_dict['rate_1st_serve_points']=rate_1st_serve_points.text
    rate_1st_serve_points_df = rate_1st_serve_points_df.append(rate_1st_serve_points_dict, ignore_index=True)

    rate_2nd_serve_points=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[6]')
    rate_2nd_serve_points_dict['rate_2nd_serve_points']=rate_2nd_serve_points.text
    rate_2nd_serve_points_df = rate_2nd_serve_points_df.append(rate_2nd_serve_points_dict, ignore_index=True)

    rate_service_game_won=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[7]')
    rate_service_game_won_dict['rate_service_game_won']=rate_service_game_won.text
    rate_service_game_won_df = rate_service_game_won_df.append(rate_service_game_won_dict, ignore_index=True)

    avg_aces_per_match=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[8]')
    avg_aces_per_match_dict['avg_aces_per_match']=avg_aces_per_match.text
    avg_aces_per_match_df = avg_aces_per_match_df.append(avg_aces_per_match_dict, ignore_index=True)

    avg_double_faults_per_match=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[9]')
    avg_double_faults_per_match_dict['avg_double_faults_per_match']=avg_double_faults_per_match.text
    avg_double_faults_per_match_df = avg_double_faults_per_match_df.append(avg_double_faults_per_match_dict, ignore_index=True)
    
    i=i+1

In [160]:
# Concat all columns
serve_leaderboard_df=pd.concat([player_df,serve_rating_df,rate_1st_serve_df,rate_1st_serve_points_df,rate_2nd_serve_points_df,
                                rate_service_game_won_df,avg_aces_per_match_df,avg_double_faults_per_match_df], axis=1)

In [161]:
serve_leaderboard_df

,player,serve_rating,rate_1st_serve,rate_1st_serve_points,rate_2nd_serve_points,rate_service_game_won,avg_aces_per_match,avg_double_faults_per_match
0,John Isner,312.1,69.2%,78.7%,56.1%,91.9%,18.5,2.3
1,Ivo Karlovic,309.8,65.5%,82.7%,53.3%,92.0%,19.8,3.5
2,Reilly Opelka,304.6,64.1%,79.4%,55.0%,90.4%,18.6,2.9
3,Milos Raonic,302.8,62.9%,81.4%,55.1%,91.2%,15.5,3.3
4,Andy Roddick,299.9,64.8%,79.3%,56.0%,90.1%,11.7,2.0
...,...,...,...,...,...,...,...,...
738,Razvan Sabau,226.2,59.3%,62.0%,44.5%,62.1%,2.4,4.1
739,Marian Vajda,223.4,54.3%,61.5%,47.6%,61.7%,2.3,4.0
740,Daniel Elsner,222.4,54.1%,63.5%,44.9%,61.8%,2.7,4.6
741,Bart Wuyts,221.6,70.1%,57.7%,40.9%,56.7%,0.4,4.2


Return Leaders Versus All Players On All Surfaces For Career

In [184]:
from fake_useragent import UserAgent

ua = UserAgent()
userAgent = ua.chrome

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument(f'user-agent={userAgent}')

browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/mariahostiananapitupulu/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_1423/2052930819.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [185]:
# Define web pages
leaderboard_return_leader = 'https://www.atptour.com/en/stats/leaderboard?boardType=return&timeFrame=Career&surface=all&versusRank=all&formerNo1=false'

In [186]:
# Open browser serve leaderboard
browser.get(leaderboard_return_leader)

In [187]:
# Define class for serve leaderboard
all_data=browser.find_element(by=By.XPATH, value='/html/body/div[3]/div[2]/div[1]/div/div[1]/div[2]/div[3]/div/div[1]/div/table/tbody')
players=all_data.find_elements(by=By.CLASS_NAME, value='player-leaderboard-info')

In [188]:
# Create dataframe
player_dict=dict()
return_rating_dict=dict()
rate_1st_return_points_dict=dict()
rate_2nd_return_points_dict=dict()
return_game_won_dict=dict()
break_point_dict=dict()

player_df = pd.DataFrame()
return_rating_df = pd.DataFrame()
rate_1st_return_points_df=pd.DataFrame()
rate_2nd_return_points_df=pd.DataFrame()
return_game_won_df=pd.DataFrame()
break_point_df=pd.DataFrame()

In [189]:
# Fetch all data

i=1

for player in players:
    player_dict['player']=player.text
    player_df = player_df.append(player_dict, ignore_index=True)
    
    return_rating=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[3]')
    return_rating_dict['return_rating']=return_rating.text
    return_rating_df = return_rating_df.append(return_rating_dict, ignore_index=True)

    rate_1st_return_points=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[4]')
    rate_1st_return_points_dict['rate_1st_return_points']=rate_1st_return_points.text
    rate_1st_return_points_df = rate_1st_return_points_df.append(rate_1st_return_points_dict, ignore_index=True)

    rate_2nd_return_points=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[5]')
    rate_2nd_return_points_dict['rate_2nd_return_points']=rate_2nd_return_points.text
    rate_2nd_return_points_df = rate_2nd_return_points_df.append(rate_2nd_return_points_dict, ignore_index=True)

    return_game_won=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[6]')
    return_game_won_dict['return_game_won']=return_game_won.text
    return_game_won_df = return_game_won_df.append(return_game_won_dict, ignore_index=True)

    break_point=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[7]')
    break_point_dict['break_point']=break_point.text
    break_point_df = break_point_df.append(break_point_dict, ignore_index=True)

    i=i+1

In [196]:
# Concat all columns
return_leaderboard_df=pd.concat([player_df,return_rating_df,rate_1st_return_points_df,rate_2nd_return_points_df,rate_2nd_serve_points_df,
                                return_game_won_df,break_point_df], axis=1)

In [197]:
return_leaderboard_df

,player,return_rating,rate_1st_return_points,rate_2nd_return_points,rate_2nd_serve_points,return_game_won,break_point
0,Guillermo Coria,171.5,36.1%,54.4%,56.1%,35.3%,45.7%
1,Rafael Nadal,168.2,34.2%,55.4%,53.3%,33.6%,45.0%
2,Guillermo Perez-Roldan,168.1,35.8%,53.2%,55.0%,32.5%,46.6%
3,Alberto Berasategui,166.6,33.6%,56.3%,55.1%,32.4%,44.3%
4,Franco Davin,166.3,34.8%,53.2%,56.0%,32.8%,45.5%
...,...,...,...,...,...,...,...
738,John Isner,104.6,22.4%,41.8%,44.5%,10.2%,30.2%
739,Wayne Arthurs,103.9,23.4%,40.5%,47.6%,10.0%,30.0%
740,Reilly Opelka,103.7,20.5%,42.1%,44.9%,10.0%,31.1%
741,Ivo Karlovic,100.1,21.2%,40.2%,40.9%,8.5%,30.2%


Under Pressure Leaders Versus All Players On All Surfaces For Career

https://www.atptour.com/en/stats/leaderboard?boardType=pressure&timeFrame=Career&surface=all&versusRank=all&formerNo1=false

In [200]:
from fake_useragent import UserAgent

ua = UserAgent()
userAgent = ua.chrome

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument(f'user-agent={userAgent}')

browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/mariahostiananapitupulu/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
/var/folders/ff/hnk1h5h50t944b7pr_w5mpdr0000gn/T/ipykernel_1423/2052930819.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [201]:
# Define web pages
leaderboard_pressure_leader = 'https://www.atptour.com/en/stats/leaderboard?boardType=pressure&timeFrame=Career&surface=all&versusRank=all&formerNo1=false'

In [202]:
# Open browser serve leaderboard
browser.get(leaderboard_pressure_leader)

In [203]:
# Define class for serve leaderboard
all_data=browser.find_element(by=By.XPATH, value='/html/body/div[3]/div[2]/div[1]/div/div[1]/div[2]/div[3]/div/div[1]/div/table/tbody')
players=all_data.find_elements(by=By.CLASS_NAME, value='stats-player-name')

In [204]:
# Create dataframe
player_dict=dict()
pressure_rating_dict=dict()
rate_break_points_converted_dict=dict()
rate_break_points_saved_dict=dict()
rate_tie_break_dict=dict()
rate_deciding_set_dict=dict()

player_df = pd.DataFrame()
pressure_rating_df = pd.DataFrame()
rate_break_points_converted_df=pd.DataFrame()
rate_break_points_saved_df=pd.DataFrame()
rate_tie_break_df=pd.DataFrame()
rate_deciding_set_df=pd.DataFrame()

In [205]:
# Fetch all data

i=1

for player in players:
    player_dict['player']=player.text
    player_df = player_df.append(player_dict, ignore_index=True)
    
    pressure_rating=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[3]')
    pressure_rating_dict['pressure_rating']=pressure_rating.text
    pressure_rating_df = pressure_rating_df.append(pressure_rating_dict, ignore_index=True)

    rate_break_points_converted=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[4]')
    rate_break_points_converted_dict['rate_break_points_converted']=rate_break_points_converted.text
    rate_break_points_converted_df = rate_break_points_converted_df.append(rate_break_points_converted_dict, ignore_index=True)

    rate_break_points_saved=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[5]')
    rate_break_points_saved_dict['rate_break_points_saved']=rate_2nd_return_points.text
    rate_break_points_saved_df = rate_break_points_saved_df.append(rate_break_points_saved_dict, ignore_index=True)

    rate_tie_break=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[6]')
    rate_tie_break_dict['rate_tie_break']=rate_tie_break.text
    rate_tie_break_df = rate_tie_break_df.append(rate_tie_break_dict, ignore_index=True)

    rate_deciding_set=browser.find_element(by=By.XPATH, value='//*[@id="leaderboardTable"]/tr['+str(i)+']/td[7]')
    rate_deciding_set_dict['rate_deciding_set']=break_point.text
    rate_deciding_set_df = rate_deciding_set_df.append(rate_deciding_set_dict, ignore_index=True)

    i=i+1

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: headless chrome=100.0.4896.75)
Stacktrace:
0   chromedriver                        0x00000001006373d9 chromedriver + 5104601
1   chromedriver                        0x00000001005c7bf3 chromedriver + 4647923
2   chromedriver                        0x00000001001b8b48 chromedriver + 392008
3   chromedriver                        0x00000001001a3c12 chromedriver + 306194
4   chromedriver                        0x00000001001a2b6a chromedriver + 301930
5   chromedriver                        0x00000001001a306c chromedriver + 303212
6   chromedriver                        0x00000001001c2a06 chromedriver + 432646
7   chromedriver                        0x00000001001bb498 chromedriver + 402584
8   chromedriver                        0x00000001001bb19e chromedriver + 401822
9   chromedriver                        0x00000001001bb8d2 chromedriver + 403666
10  chromedriver                        0x000

In [ ]:
player_df

In [ ]:
pressure_rating_df

In [ ]:
rate_break_points_converted_df

In [ ]:
rate_break_points_saved_df

In [ ]:
rate_tie_break_df

In [ ]:
rate_deciding_set_df